# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the cities csv into DF
vacationDF = pd.read_csv('Instructions/output_data/cities.csv')
vacationDF = vacationDF.drop(columns=['Unnamed: 0'])
vacationDF.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,vila velha,-20.3297,-40.2925,78.75,69,40,14.97,BR,1635016282
1,kahului,20.8947,-156.4700,81.66,78,40,14.97,US,1635016283
2,ponta do sol,32.6667,-17.1000,70.43,53,24,4.52,PT,1635016283
3,rikitea,-23.1203,-134.9692,74.77,82,96,20.58,PF,1635016283
4,bathsheba,13.2167,-59.5167,86.61,66,40,21.85,BB,1635016284


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations
locations = vacationDF[['Lat', 'Lng']].astype(float)

# Store humidity as weight
weight = vacationDF['Humidity'].astype(float)

# Create a Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=100, point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Drop null values
vacationDF = vacationDF.dropna()

perfectVaca = vacationDF.loc[(vacationDF['Max Temp'] < 90) & (vacationDF['Max Temp'] > 75)]
perfectVaca = perfectVaca.loc[perfectVaca['Cloudiness'] == 0]
perfectVaca = perfectVaca.loc[perfectVaca['Wind Speed'] < 10]
perfectVaca

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
75,avera,33.1940,-82.5271,76.51,38,0,4.16,US,1635016305
102,salalah,17.0151,54.0924,77.09,73,0,3.44,OM,1635016313
222,riyadh,24.6877,46.7219,80.74,23,0,9.17,SA,1635016350
295,tombouctou,20.0000,-3.0000,86.74,20,0,5.64,ML,1635016371
351,abu dhabi,24.4667,54.3667,86.11,70,0,5.75,AE,1635016112
432,atar,20.5169,-13.0499,86.05,12,0,6.82,MR,1635016410
442,chiredzi,-21.0500,31.6667,88.75,27,0,6.44,ZW,1635016413
451,dubai,25.2582,55.3047,87.73,55,0,4.61,AE,1635016229
507,buraydah,26.3260,43.9750,80.28,16,0,7.81,SA,1635016431


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Store into variable named hotel_df
hotel_df = perfectVaca.copy()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
75,avera,33.1940,-82.5271,76.51,38,0,4.16,US,1635016305
102,salalah,17.0151,54.0924,77.09,73,0,3.44,OM,1635016313
222,riyadh,24.6877,46.7219,80.74,23,0,9.17,SA,1635016350
295,tombouctou,20.0000,-3.0000,86.74,20,0,5.64,ML,1635016371
351,abu dhabi,24.4667,54.3667,86.11,70,0,5.75,AE,1635016112
432,atar,20.5169,-13.0499,86.05,12,0,6.82,MR,1635016410
442,chiredzi,-21.0500,31.6667,88.75,27,0,6.44,ZW,1635016413
451,dubai,25.2582,55.3047,87.73,55,0,4.61,AE,1635016229
507,buraydah,26.3260,43.9750,80.28,16,0,7.81,SA,1635016431


In [6]:
# Add a "Hotel Name" column to DF
hotel_df["Hotel Name"] = ""

In [7]:
# URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key,
}

# Hit the Google Places API for each city's coordinates.
for index, row in hotel_df.iterrows():
    # get lat, lng from DF
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # make request and convert to json
    response = requests.get(base_url, params=params).json()
    #pprint(response)

    # extract results
    results = response['results']
    
    # Store the first Hotel result into the DataFrame.
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        
    except:
        print("Missing field/result... skipping.")
# Plot markers on top of the heatmap.

Missing field/result... skipping.
Missing field/result... skipping.


In [10]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
75,avera,33.1940,-82.5271,76.51,38,0,4.16,US,1635016305,
102,salalah,17.0151,54.0924,77.09,73,0,3.44,OM,1635016313,Muscat International Hotel plaza Salalah
222,riyadh,24.6877,46.7219,80.74,23,0,9.17,SA,1635016350,"Radisson Blu Hotel, Riyadh"
295,tombouctou,20.0000,-3.0000,86.74,20,0,5.64,ML,1635016371,
351,abu dhabi,24.4667,54.3667,86.11,70,0,5.75,AE,1635016112,Emirates Palace
432,atar,20.5169,-13.0499,86.05,12,0,6.82,MR,1635016410,Auberge et Camping Inimi
442,chiredzi,-21.0500,31.6667,88.75,27,0,6.44,ZW,1635016413,Palm Tree Place
451,dubai,25.2582,55.3047,87.73,55,0,4.61,AE,1635016229,Le Meridien Dubai Hotel & Conference Centre
507,buraydah,26.3260,43.9750,80.28,16,0,7.81,SA,1635016431,Best Western Plus Buraidah


In [14]:
# drop cities without hotels
nan_value = float("Nan")

hotel_df.replace("", nan_value, inplace=True)

hotel_df.dropna(subset = ["Hotel Name"], inplace=True)

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
102,salalah,17.0151,54.0924,77.09,73,0,3.44,OM,1635016313,Muscat International Hotel plaza Salalah
222,riyadh,24.6877,46.7219,80.74,23,0,9.17,SA,1635016350,"Radisson Blu Hotel, Riyadh"
351,abu dhabi,24.4667,54.3667,86.11,70,0,5.75,AE,1635016112,Emirates Palace
432,atar,20.5169,-13.0499,86.05,12,0,6.82,MR,1635016410,Auberge et Camping Inimi
442,chiredzi,-21.0500,31.6667,88.75,27,0,6.44,ZW,1635016413,Palm Tree Place
451,dubai,25.2582,55.3047,87.73,55,0,4.61,AE,1635016229,Le Meridien Dubai Hotel & Conference Centre
507,buraydah,26.3260,43.9750,80.28,16,0,7.81,SA,1635016431,Best Western Plus Buraidah


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))